In [ ]:
# | default_exp lobby

In [ ]:
# | export
from typing import TypeVar, Generic, Type
import datetime as dt
import logging
from dataclasses import field
import random, string
import fastcore.all as fc
from fasthtml.common import *
from meme_games.utils import *
from meme_games.user import *

In [ ]:
#| export
logger = logging.getLogger(__name__)

In [ ]:
# | export

@dataclass
class BaseLobbyMember:
    user: User
    ws: Optional[FunctionType] = None
    joined_at: dt.datetime = field(default_factory=dt.datetime.now)
    score: int = 0

    def __getattr__(self, name): return getattr(self.user, name)
    def connect(self, ws: FunctionType): self.ws = ws
    def disconnect(self): self.ws = None
    def add_score(self, score: int): self.score += score
    def reset_score(self): self.score = 0
    def sync_user(self, u: User): self.user = u

    @property
    def is_connected(self): return self.ws is not None

In [ ]:
# | export

TMember = TypeVar('TMember', bound=BaseLobbyMember)


@dataclass
class BaseLobby(Generic[TMember]):
    id: str
    host: TMember
    members: dict[str, TMember] = field(default_factory=dict)
    last_active: dt.datetime = field(default_factory=dt.datetime.now)

    def __post_init__(self):
        if self.host.uid not in self.members:
            self.members[self.host.uid] = self.host

    @classmethod
    def create_member(cls, user: User, ws: FunctionType = None, **kwargs):
        return BaseLobbyMember(user, ws=ws, **kwargs)
    
    def add_member(self, user: User, ws: FunctionType = None, **kwargs):
        self.last_active = dt.datetime.now()
        m = self.create_member(user, ws=ws, **kwargs)
        self.members[user.uid] = m
        return m

    def get_member(self, uid: str) -> Optional[TMember]:
        self.last_active = dt.datetime.now()
        return self.members.get(uid)

    @fc.delegates(create_member)
    def get_or_create_member(self, user: User, **kwargs):
        self.last_active = dt.datetime.now()
        u = self.members.setdefault(user.uid, self.create_member(user, **kwargs))
        return u

    @classmethod
    def new(cls, id: str, host: User) -> 'BaseLobby':
        host = cls.create_member(host)
        return cls(id, host)

class Lobby(BaseLobby[BaseLobbyMember]): pass

In [ ]:
# | export

TLobby = TypeVar('TLobby', bound=BaseLobby)

class LobbyManager(Generic[TLobby]):
    lobby_lifetime = dt.timedelta(days=1)

    def __init__(self):
        self.lobbies = {}

    def __repr__(self): return f'{self.__class__.__name__}(active_lobbies={len(self.lobbies)})'

    def create_lobby(self, u: User, lobby_cls: TLobby, lobby_id: Optional[str] = None):
        lobby_id = lobby_id or random_id()
        while lobby_id in self.lobbies: lobby_id = random_id()
        lobby = lobby_cls.new(lobby_id, u)
        self.lobbies[lobby_id] = lobby
        return lobby

    def get_lobby(self, id: Optional[str] = None, lobby_cls: Type[TLobby] = BaseLobby) -> Optional[Lobby]:
        lobby = self.lobbies.get(id)
        if lobby and isinstance(lobby, lobby_cls): return lobby
        return
    
    def delete_lobby(self, id: str): self.lobbies.pop(id)

    def get_or_create(self, u: User, id: Optional[str] = None, lobby_cls: TLobby = BaseLobby) -> tuple[Lobby, bool]:
        '''Returns the lobby if it exists, otherwise creates one if id was valid. Returns (lobby, created)'''
        if not id or not id.isascii(): raise HTTPException(400, 'Invalid lobby id') 
        if lobby := self.get_lobby(id, lobby_cls): return lobby, False
        return self.create_lobby(u, lobby_cls, id), True

In [ ]:
db = init_db()
user_manager = UserManager(db)
user = user_manager.create()
manager = LobbyManager()

In [ ]:
lobby = manager.create_lobby(user)

## Views

In [ ]:
# | export
def Members(lobby: BaseLobby):
    return Table(
        Tr(Th('Name'), Th('id'), Th('Is connected')),
        *[Tr(Td(UserName(member.user)), Td(member.uid), Td(member.is_connected)) for member in lobby.members.values()],
        id='members'
    )